Antoni Radomiński-Lasek, 407290
### Przekształcenie danych:

In [169]:
import pandas as pd
import datetime
import numpy as np

data = pd.read_csv('../Oryginal_Data/weather_to_work.csv', sep = ';')
# usuwam kolumny które były separatorami
data = data.drop(columns=data.columns[5::2])
# zmieniam nazwy kolumn i dodatkowo pierwszą kolumnę muszę później dodać
col_names = ['id', 'year', 'month', 'element', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'd9', 'd10', 'd11', 'd12', 'd13', 'd14', 'd15' ,'d16', 'd17', 'd18', 'd19', 'd20', 'd21', 'd22', 'd23', 'd24', 'd25', 'd26', 'd27', 'd28', 'd29', 'd30', 'd31']
first_row = data.columns
data.columns = col_names
data = data.append(pd.Series(first_row, index=data.columns, name=1713))
data['year'] = data['year'].astype(int)
data['month'] = data['month'].astype(int)
data = data.sort_values(['year', 'month', 'element'], ignore_index=True, ascending = [True, True, False])
data
# Otrzymałem oczyszczone dane i mogę przystąpić do ich pożądkowania

C:\Users\anton\AppData\Local\Temp\ipykernel_25948\2383969421.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(pd.Series(first_row, index=data.columns, name=1713))


,id,year,month,element,d1,d2,d3,d4,d5,d6,...,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
0,MX000017004,1955,4,TMIN,150,150,160,150,160,160,...,170,170,170,180,190,190,170,180,160,-9999
1,MX000017004,1955,4,TMAX,310,310.1,310.2,320,330,320.1,...,330.5,330.6,330.7,330.8,330.9,330.10,340,330.11,320.9,-9999
2,MX000017004,1955,4,PRCP,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,-9999
3,MX000017004,1955,5,TMIN,200,160,160,150,150,150,...,170,190,190,190,180,160,150,170,150,160
4,MX000017004,1955,5,TMAX,310,310,310,300,300,300,...,330,340,350,330,310,310,320,310,300,290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1709,MX000017004,2011,3,TMAX,-9999,-9999,-9999,-9999,332,-9999,...,-9999,-9999,-9999,-9999,332,-9999,-9999,-9999,-9999,365
1710,MX000017004,2011,3,PRCP,0,-9999,0,0,0,-9999,...,0,-9999,0,-9999,0,-9999,-9999,0,-9999,0
1711,MX000017004,2011,4,TMIN,-9999,168,-9999,-9999,-9999,-9999,...,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999
1712,MX000017004,2011,4,TMAX,-9999,350,-9999,-9999,-9999,-9999,...,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999


In [170]:
# Pierwszym etapem będzie przeprowadzenie MELTINGU
data_1 = pd.melt(data, id_vars=['id', 'year', 'month', 'element'], var_name='day')
data_1['value'] = data_1['value'].astype(float)

In [171]:
# Usuwamy niepotrzebne kolumny
data_1 = data_1[data_1.value != -9999]
data_1 = data_1[data_1.value != np.NaN]
data_1 = data_1[data_1.value != 0]
data_1 = data_1.sort_values(['year', 'month', 'day', 'element'], ignore_index=True, ascending = [True, True, True, False])

In [172]:
df = data_1

data_1["day"] = data_1["day"].str.extract("d(\d+)", expand=False)  
data_1["id"] = "MX17004"

# Zmiana na numeryczne wartości
data_1[["year","month","day"]] = data_1[["year","month","day"]].apply(lambda x: pd.to_numeric(x, errors='ignore'))

# Łączę w jedną kolumnę daty
def create_date_from_year_month_day(row):
    return datetime.datetime(year=row["year"], month=int(row["month"]), day=row["day"])

data_1["date"] = data_1.apply(lambda row: create_date_from_year_month_day(row), axis=1)
data_1 = data_1.drop(['year',"month","day"], axis=1)
data_1 = data_1.dropna()

In [173]:
# Daje tmin i tmax jako kolumny
data_1 = data_1.pivot_table(index=["id","date"], columns="element", values="value")
data_1 = data_1.drop(["PRCP"], axis=1)
data_1.reset_index(drop=False, inplace=True)

In [174]:
# Zapisuję plik do folderu Analisis Data i wyświetlam w ramach sprawdzenia danych
data_1.to_csv('../Analysis Data/clear_weather.csv')
data_1

element,id,date,TMAX,TMIN
0,MX17004,1955-04-01,310.0,150.0
1,MX17004,1955-04-02,310.1,150.0
2,MX17004,1955-04-03,310.2,160.0
3,MX17004,1955-04-04,320.0,150.0
4,MX17004,1955-04-05,330.0,160.0
...,...,...,...,...
13005,MX17004,2011-03-05,332.0,148.0
13006,MX17004,2011-03-10,320.0,127.0
13007,MX17004,2011-03-26,332.0,162.0
13008,MX17004,2011-03-31,365.0,170.0
